# 使用Claude Sonnet 4.5进行记忆和上下文管理

学习如何使用Claude的记忆工具和上下文编辑功能构建跨对话学习和改进的AI代理。

## 目录

1. [介绍：为什么记忆很重要](#introduction)
2. [使用案例](#use-cases)
3. [快速开始示例](#quick-start)
4. [工作原理](#how-it-works)
5. [代码审查助手演示](#demo)
6. [实际应用](#real-world)
7. [最佳实践](#best-practices)

## 设置

### 对于VSCode用户

```bash
# 1. 创建虚拟环境
python -m venv .venv

# 2. 激活它
source .venv/bin/activate  # macOS/Linux
# 或者: .venv\Scripts\activate  # Windows

# 3. 安装依赖
pip install -r requirements.txt

# 4. 在VSCode中选择.venv作为内核（右上角）
```

### API密钥

```bash
cp .env.example .env
# 编辑.env并添加您的ANTHROPIC_API_KEY
```

从以下网址获取您的API密钥：https://console.anthropic.com/

## 1. 介绍：为什么记忆很重要 {#introduction}

本教程演示了[AI代理的有效上下文工程](https://www.anthropic.com/engineering/effective-context-engineering-for-ai-agents)一文中描述的上下文工程模式的实际实现。该文涵盖了上下文是有限资源的原因、注意力预算如何工作，以及构建有效代理的策略——您将在这里看到这些技术的实际应用。

### 问题

大语言模型有有限的上下文窗口（Claude 4为200k个标记）。虽然这看起来很大，但会出现几个挑战：

- **上下文限制**：长对话或复杂任务可能超出可用上下文
- **计算成本**：处理大上下文很昂贵——注意力机制呈二次方扩展
- **重复模式**：跨对话的类似任务每次都需要重新解释上下文
- **信息丢失**：当上下文填满时，较早的重要信息会丢失

### 解决方案

Claude Sonnet 4.5引入了两个强大的功能：

1. **记忆工具** (`memory_20250818`)：实现跨对话学习
   - Claude可以写下它学到的东西以供将来参考
   - `/memories`目录下的基于文件的系统
   - 客户端实现让您完全控制

2. **上下文编辑** (`clear_tool_uses_20250919`)：自动管理上下文
   - 当上下文增长较大时清除旧工具结果
   - 保留最近上下文同时保留记忆
   - 可配置的触发器和保留策略

### 好处

构建**随着时间推移在您的特定任务上变得更出色**的AI代理：

- **会话1**：Claude解决问题，记录模式
- **会话2**：Claude立即应用学到的模式（更快！）
- **长会话**：上下文编辑保持对话可控

把它想象成给Claude一个笔记本做笔记和参考——就像人类做的那样。

## 2. 使用案例 {#use-cases}

记忆和上下文管理可以实现强大的新工作流程：

### 🔍 代码审查助手
- 从过去的审查中学习调试模式
- 在未来的会话中立即识别类似的错误
- 建立团队特定的代码质量知识
- **生产就绪**：与[claude-code-action](https://github.com/anthropics/claude-code-action)集成进行GitHub PR审查

### 📚 研究助手
- 在多个会话中积累主题知识
- 连接不同研究线索的见解
- 维护参考文献和来源跟踪

### 💬 客户支持机器人
- 学习用户偏好和沟通风格
- 记住常见问题和解决方案
- 从交互中建立产品知识库

### 📊 数据分析助手
- 记住数据集模式和异常
- 存储效果良好的分析技术
- 随着时间推移建立领域特定见解

**支持的模型**：Claude Opus 4.1 (`claude-opus-4-1`) 和 Claude Sonnet 4.5 (`claude-sonnet-4-5`)

**本教程侧重于代码审查助手**，因为它清楚地展示了记忆（学习模式）和上下文编辑（处理长审查）。

## 3. 快速开始示例 {#quick-start}

让我们通过简单示例看看记忆和上下文管理的实际应用。

### 设置

首先，安装依赖项并配置您的环境：

In [1]:
# Install required packages
# Option 1: From requirements.txt
# %pip install -q -r requirements.txt

# Option 2: Direct install
%pip install -q anthropic python-dotenv ipykernel


[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


**⚠️ 重要**：在此目录中创建`.env`文件：

```bash
# 将.env.example复制到.env并添加您的API密钥
cp .env.example .env
```

然后编辑`.env`以从https://console.anthropic.com/添加您的Anthropic API密钥

In [2]:
import os
from typing import cast

from anthropic import Anthropic
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

API_KEY = os.getenv("ANTHROPIC_API_KEY")
MODEL = os.getenv("ANTHROPIC_MODEL")

if not API_KEY:
    raise ValueError("ANTHROPIC_API_KEY not found. Copy .env.example to .env and add your API key.")

if not MODEL:
    raise ValueError("ANTHROPIC_MODEL not found. Copy .env.example to .env and set the model.")

MODEL = cast(str, MODEL)

client = Anthropic(api_key=API_KEY)

print("✓ API key loaded")
print(f"✓ Using model: {MODEL}")

✓ API key loaded
✓ Using model: claude-sonnet-4-5


### 示例1：基本记忆使用

让我们看看Claude如何使用记忆来存储信息以供将来参考。

**辅助函数**

这些示例使用来自`demo_helpers.py`的辅助函数：

- **`run_conversation_loop()`**：处理API对话循环
  - 调用启用记忆工具的Claude的API
  - 执行工具使用（记忆操作）
  - 继续直到Claude停止使用工具
  - 返回最终响应

- **`run_conversation_turn()`**：单次回合（在示例3中使用）
  - 与上面相同，但在一次API调用后返回
  - 当您需要细粒度控制时很有用

- **`print_context_management_info()`**：显示上下文清除统计信息
  - 显示保存的标记、清除的工具使用
  - 有助于可视化上下文编辑何时触发

**⚠️ 关于记忆清除的说明**

以下单元格清除所有记忆文件以为此演示提供干净的环境。这对于多次运行笔记本以查看一致结果很有用。

**在生产应用程序中**，您应该仔细考虑是否清除所有记忆，因为它会永久删除学到的模式。考虑使用选择性删除或将记忆组织到项目特定目录中。

In [3]:
# Import helper functions
from memory_demo.demo_helpers import (
    run_conversation_loop,
    run_conversation_turn,
    print_context_management_info,
)
from memory_tool import MemoryToolHandler

# Initialize
client = Anthropic(api_key=API_KEY)
memory = MemoryToolHandler(base_path="./demo_memory")

# Clear any existing memories to start fresh
print("🧹 Clearing previous memories...")
memory.clear_all_memory()
print("✓ Memory cleared\n")

# Load example code with a race condition bug
with open("memory_demo/sample_code/web_scraper_v1.py", "r") as f:
    code_to_review = f.read()

messages = [
    {
        "role": "user",
        "content": f"I'm reviewing a multi-threaded web scraper that sometimes returns fewer results than expected. The count is inconsistent across runs. Can you find the issue?\n\n```python\n{code_to_review}\n```",
    }
]

print("=" * 60)
print("📝 SESSION 1: Learning from a bug")
print("=" * 60)

# Run conversation loop
response = run_conversation_loop(
    client=client,
    model=MODEL,
    messages=messages,
    memory_handler=memory,
    system="You are a code reviewer.",
    max_tokens=2048,
    max_turns=5,
    verbose=True,
)

print("\n" + "=" * 60)
print("✅ Session 1 complete!")
print("=" * 60)

🧹 Clearing previous memories...
✓ Memory cleared

📝 SESSION 1: Learning from a bug

🔄 Turn 1:
💬 Claude: I'll review this multi-threaded web scraper for you. Let me first check my memory and then analyze the code.

  🔧 Memory tool: view /memories
  ✓ Result: Directory: /memories
(empty)

🔄 Turn 2:
  🔧 Memory tool: create /memories/review_progress.md
  ✓ Result: File created successfully at /memories/review_progress.md

🔄 Turn 3:
💬 Claude: 

## Code Review: Multi-threaded Web Scraper - Race Condition Issues

Great catch on identifying this as a concurrency issue! I've found **multiple critical race conditions** that explain the inconsistent results.

### 🔴 **Critical Issues**

#### **1. Race Condition in `self.results` (Primary Bug)**
**Location:** Line in `scrape_urls()` method
```python
self.results.append(result)  # RACE CONDITION
```

**Problem:** 
- Python's `list.append()` is **NOT thread-safe** for concurrent modifications
- Multiple threads simultaneously appending can cause:
  -

**发生了什么？**

1. Claude检查了它的记忆（首次运行时为空）
2. 识别了错误：**竞态条件** - 多个线程在没有同步的情况下修改共享状态（`self.results`和`self.failed_urls`）
3. 将并发模式存储在记忆中以供将来参考

现在让我们看看神奇之处 - Claude在**新对话**中应用这个学到的模式：

### 示例2：跨对话学习

开始一个全新的对话 - 记忆持续存在！

In [4]:
# NEW conversation (empty messages)
# Load API client code with similar concurrency issue
with open("memory_demo/sample_code/api_client_v1.py", "r") as f:
    code_to_review = f.read()

messages = [
    {"role": "user", "content": f"Review this API client code:\n\n```python\n{code_to_review}\n```"}
]

print("=" * 60)
print("🚀 SESSION 2: Applying learned pattern")
print("=" * 60)

# Run conversation loop
response = run_conversation_loop(
    client=client,
    model=MODEL,
    messages=messages,
    memory_handler=memory,
    system="You are a code reviewer.",
    max_tokens=2048,
    max_turns=5,
    verbose=True,
)

print("\n" + "=" * 60)
print("✅ Session 2 complete!")
print("=" * 60)

🚀 SESSION 2: Applying learned pattern

🔄 Turn 1:
  🔧 Memory tool: view /memories
  ✓ Result: Directory: /memories
- review_progress.md

🔄 Turn 2:
  🔧 Memory tool: view /memories/review_progress.md
  ✓ Result:    1: # Code Review: Multi-threaded Web Scraper
   2: 
   3: ## Task
   4: Revie...

🔄 Turn 3:
  🔧 Memory tool: str_replace /memories/review_progress.md
  ✓ Result: File /memories/review_progress.md has been edited successfully

🔄 Turn 4:
💬 Claude: 

# Code Review: Async API Client

## 🔴 Critical Issues

### 1. **Race Condition on Shared State (High Severity)**
**Location:** `fetch_all()` method, lines with `self.responses.append()` and `self.error_count += 1`

**Problem:** 
Multiple coroutines concurrently modify shared instance variables without synchronization. While the comments correctly identify this as an issue, there's an important nuance: Python's asyncio is **single-threaded**, so the GIL isn't the protective factor here—the real issue is that coroutines can be suspended

**注意差异：**

- Claude**立即检查了记忆**并找到了线程安全/并发模式
- **立即识别**了异步代码中的类似问题，无需重新学习
- 响应**更快**，因为它应用了关于共享可变状态的存储知识

这是**跨对话学习**的实际应用！

### 示例3：在保留记忆的同时清除上下文

在包含许多代码文件的**长审查会话**期间会发生什么？

- 上下文充满了来自先前审查的工具结果
- 但记忆（学到的模式）必须持续存在！

让我们触发**上下文编辑**以查看Claude如何自动管理这一点。

**关于配置的说明：**我们使用`clear_at_least: 50`个标记，因为记忆工具操作的结果很小（约50-150个标记）。在生产环境中使用更大的工具结果（如网络搜索或代码执行），您将使用3000-5000个标记等更高值。

In [5]:
# Configure context management to clear aggressively for demo
CONTEXT_MANAGEMENT = {
    "edits": [
        {
            "type": "clear_tool_uses_20250919",
            "trigger": {
                "type": "input_tokens",
                "value": 5000,
            },  # Lower threshold to trigger clearing sooner
            "keep": {"type": "tool_uses", "value": 1},  # Keep only the last tool use
            "clear_at_least": {"type": "input_tokens", "value": 50},
        }
    ]
}

# Continue from previous session - memory persists!
# Add multiple code reviews to build up context

print("=" * 60)
print("📚 SESSION 3: Long review session with context clearing")
print("=" * 60)
print()

# Review 1: Data processor (larger file)
with open("memory_demo/sample_code/data_processor_v1.py", "r") as f:
    data_processor_code = f.read()

messages.extend(
    [
        {
            "role": "user",
            "content": f"Review this data processor:\n\n```python\n{data_processor_code}\n```",
        }
    ]
)

print("📝 Review 1: Data processor")
response = run_conversation_turn(
    client=client,
    model=MODEL,
    messages=messages,
    memory_handler=memory,
    system="You are a code reviewer.",
    context_management=CONTEXT_MANAGEMENT,
    max_tokens=2048,
    verbose=True,
)

# Add response to messages
messages.append({"role": "assistant", "content": response[1]})
if response[2]:
    messages.append({"role": "user", "content": response[2]})

print(f"  📊 Input tokens: {response[0].usage.input_tokens:,}")
context_cleared, saved = print_context_management_info(response[0])
print()

# Review 2: Add SQL code
with open("memory_demo/sample_code/sql_query_builder.py", "r") as f:
    sql_code = f.read()

messages.extend(
    [{"role": "user", "content": f"Review this SQL query builder:\n\n```python\n{sql_code}\n```"}]
)

print("📝 Review 2: SQL query builder")
response = run_conversation_turn(
    client=client,
    model=MODEL,
    messages=messages,
    memory_handler=memory,
    system="You are a code reviewer.",
    context_management=CONTEXT_MANAGEMENT,
    max_tokens=2048,
    verbose=True,
)

messages.append({"role": "assistant", "content": response[1]})
if response[2]:
    messages.append({"role": "user", "content": response[2]})

print(f"  📊 Input tokens: {response[0].usage.input_tokens:,}")
context_cleared, saved = print_context_management_info(response[0])
print()

print("=" * 60)
print("✅ Session 3 complete!")
print("=" * 60)

📚 SESSION 3: Long review session with context clearing

📝 Review 1: Data processor
  🔧 Memory tool: str_replace /memories/review_progress.md
  ✓ Result: File /memories/review_progress.md has been edited successfully
  📊 Input tokens: 6,243
  ✂️  Context editing triggered!
      • Cleared 1 tool uses
      • Saved 66 tokens
      • After clearing: 6,243 tokens

📝 Review 2: SQL query builder
  🔧 Memory tool: str_replace /memories/review_progress.md
  ✓ Result: File /memories/review_progress.md has been edited successfully
  📊 Input tokens: 7,471
  ✂️  Context editing triggered!
      • Cleared 1 tool uses
      • Saved 66 tokens
      • After clearing: 7,471 tokens

✅ Session 3 complete!


**刚才发生了什么？**

在多次审查过程中随着上下文增长：
1. 当输入标记超过5,000时**自动触发上下文清除**
2. **删除了旧工具结果** - 清除2个工具使用，每次节省约66个标记
3. **记忆文件保持完整** - Claude仍然可以查询学到的模式
4. **标记使用继续增长**但由于清除而增长较慢

这展示了关键好处：
- **短期记忆**（带有工具结果的对话上下文）→ 清除以节省空间
- **长期记忆**（存储在`/memories`中的模式）→ 跨会话持续

**为什么节省的标记这么少？**记忆工具操作返回紧凑结果（文件路径、成功消息）。`str_replace`操作只返回"文件编辑成功"加上元数据。在生产用例中，工具结果更大（网络搜索返回完整文章、代码执行输出较长），上下文清除将节省数千个标记。

让我们验证记忆在清除中幸存下来：

In [6]:
# Verify memory persists after context clearing
import os

print("📂 Memory files in demo_memory/:")
print()

for root, dirs, files in os.walk("./demo_memory"):
    # Calculate relative path for display
    level = root.replace("./demo_memory", "").count(os.sep)
    indent = "  " * level
    folder_name = os.path.basename(root) or "demo_memory"
    print(f"{indent}{folder_name}/")

    sub_indent = "  " * (level + 1)
    for file in files:
        file_path = os.path.join(root, file)
        size = os.path.getsize(file_path)
        print(f"{sub_indent}├── {file} ({size} bytes)")

print()
print("✅ All learned patterns preserved despite context clearing!")

📂 Memory files in demo_memory/:

demo_memory/
  memories/
    ├── review_progress.md (257 bytes)

✅ All learned patterns preserved despite context clearing!


## 4. 工作原理 {#how-it-works}

### 记忆工具架构

记忆工具是**客户端的** - 您控制存储。Claude进行工具调用，您的应用程序执行它们。

#### 记忆工具命令

| 命令 | 描述 | 示例 |
|---------|-------------|---------|
| `view` | 显示目录或文件内容 | `{"command": "view", "path": "/memories"}` |
| `create` | 创建或覆盖文件 | `{"command": "create", "path": "/memories/notes.md", "file_text": "..."}` |
| `str_replace` | 替换文件中的文本 | `{"command": "str_replace", "path": "...", "old_str": "...", "new_str": "..."}` |
| `insert` | 在行号处插入文本 | `{"command": "insert", "path": "...", "insert_line": 2, "insert_text": "..."}` |
| `delete` | 删除文件或目录 | `{"command": "delete", "path": "/memories/old.txt"}` |
| `rename` | 重命名或移动文件 | `{"command": "rename", "old_path": "...", "new_path": "..."}` |

有关带有路径验证和安全措施的完整实现，请参见`memory_tool.py`。

### 理解演示代码

`code_review_demo.py`中的关键实现细节：

```python
class CodeReviewAssistant:
    def __init__(self, memory_storage_path="./memory_storage"):
        self.client = Anthropic(api_key=API_KEY)
        self.memory_handler = MemoryToolHandler(base_path=memory_storage_path)
        self.messages = []
    
    def review_code(self, code, filename, description=""):
        # 1. 添加用户消息
        self.messages.append({...})
        
        # 2. 带有工具执行的对话循环
        while True:
            response = self.client.beta.messages.create(
                model=MODEL,
                system=self._create_system_prompt(),
                messages=self.messages,
                tools=[{"type": "memory_20250818", "name": "memory"}],
                betas=["context-management-2025-06-27"],
                context_management=CONTEXT_MANAGEMENT
            )
            
            # 3. 执行工具使用
            tool_results = []
            for content in response.content:
                if content.type == "tool_use":
                    result = self._execute_tool_use(content)
                    tool_results.append({...})
            
            # 4. 如果有工具使用则继续，否则完成
            if tool_results:
                self.messages.append({"role": "user", "content": tool_results})
            else:
                break
```

**关键模式**：只要有工具使用就继续调用API，执行它们并反馈结果。

### Claude实际学到了什么

这使得记忆变得强大 - **语义模式识别**，而不仅仅是语法：

**会话1：基于线程的Web爬虫**

```python
# 错误：竞态条件
class WebScraper:
    def __init__(self):
        self.results = []  # 共享状态！
    
    def scrape_urls(self, urls):
        with ThreadPoolExecutor() as executor:
            for future in as_completed(futures):
                self.results.append(future.result())  # 竞态条件！
```

**Claude在记忆中存储的内容**（示例文件：`/memories/concurrency_patterns/thread_safety.md`）：

当Claude遇到此模式时，它将其以下见解存储到其记忆文件中：
- **症状**：并发操作中的不一致结果
- **原因**：来自多个线程的共享可变状态（列表/字典）被修改
- **解决方案**：使用锁、线程安全数据结构，或改为返回结果
- **危险信号**：线程回调中的实例变量、未使用的锁、计数器增量

---

**会话2：异步API客户端**（新对话！）

Claude首先检查记忆，找到线程安全模式，然后：
1. **识别**异步代码中的类似模式（协程也可以交错）
2. **立即应用**解决方案（无需重新学习）
3. **根据存储知识解释**

```python
# Claude立即发现这一点：
async def fetch_all(self, endpoints):
    for coro in asyncio.as_completed(tasks):
        self.responses.append(await coro)  # 相同模式！
```

---

**为什么这很重要：**

- ❌ **语法检查器**完全错过竞态条件
- ✅ **Claude学习**架构模式并跨上下文应用它们
- ✅ **跨语言**：模式也适用于Go、Java、Rust并发
- ✅ **变得更好**：每次审查都会添加到知识库

### 示例代码文件

演示使用这些示例文件（都有并发/线程安全错误）：

- `memory_demo/sample_code/web_scraper_v1.py` - 竞态条件：线程修改共享状态
- `memory_demo/sample_code/api_client_v1.py` - 异步上下文中的类似并发错误
- `memory_demo/sample_code/data_processor_v1.py` - 长会话演示的多个并发问题

让我们看一下一个：

**`memory_demo/sample_code/web_scraper_v1.py`**

```python
"""
Concurrent web scraper with a race condition bug.
Multiple threads modify shared state without synchronization.
"""

import time
from concurrent.futures import ThreadPoolExecutor, as_completed
from typing import List, Dict

import requests


class WebScraper:
    """Web scraper that fetches multiple URLs concurrently."""

    def __init__(self, max_workers: int = 10):
        self.max_workers = max_workers
        self.results = []  # BUG: Shared mutable state accessed by multiple threads!
        self.failed_urls = []  # BUG: Another race condition!

    def fetch_url(self, url: str) -> Dict[str, any]:
        """Fetch a single URL and return the result."""
        try:
            response = requests.get(url, timeout=5)
            response.raise_for_status()
            return {
                "url": url,
                "status": response.status_code,
                "content_length": len(response.content),
            }
        except requests.exceptions.RequestException as e:
            return {"url": url, "error": str(e)}

    def scrape_urls(self, urls: List[str]) -> List[Dict[str, any]]:
        """
        Scrape multiple URLs concurrently.

        BUG: self.results is accessed from multiple threads without locking!
        This causes race conditions where results can be lost or corrupted.
        """
        with ThreadPoolExecutor(max_workers=self.max_workers) as executor:
            futures = [executor.submit(self.fetch_url, url) for url in urls]

            for future in as_completed(futures):
                result = future.result()

                # RACE CONDITION: Multiple threads append to self.results simultaneously
                if "error" in result:
                    self.failed_urls.append(result["url"])  # RACE CONDITION
                else:
                    self.results.append(result)  # RACE CONDITION

        return self.results
```

**错误**：多个线程在没有锁定的情况下修改`self.results`和`self.failed_urls`！

Claude将：
1. 识别竞态条件
2. 将模式存储在`/memories/concurrency_patterns/thread_safety.md`
3. 在会话2中将此并发模式应用于异步代码

### 演示概述

我们构建了一个完整的代码审查助手。实现位于`memory_demo/code_review_demo.py`。

**运行交互式演示：**
```bash
python memory_demo/code_review_demo.py
```

演示展示：
1. **会话1**：审查带有错误的Python代码 → Claude学习模式
2. **会话2**：审查类似代码（新对话）→ Claude应用模式
3. **会话3**：长审查会话 → 上下文编辑保持可管理

## 7. 最佳实践与安全 {#best-practices}

### 记忆管理

**应该做：**
- ✅ 存储任务相关的模式，而不是对话历史
- ✅ 使用清晰的目录结构组织
- ✅ 使用描述性文件名
- ✅ 定期审查和清理记忆

**不应该做：**
- ❌ 存储敏感信息（密码、API密钥、个人身份信息）
- ❌ 让记忆无限制增长
- ❌ 不加选择地存储所有内容

### 安全：路径遍历保护

**关键**：始终验证路径以防止目录遍历攻击。实现请参见`memory_tool.py`。

### 安全：记忆投毒

**⚠️ 关键风险**：记忆文件被读回到Claude的上下文中，使它们成为提示注入的潜在载体。

**缓解策略：**
1. **内容清理**：在存储前过滤危险模式
2. **记忆范围隔离**：每个用户/项目的隔离
3. **记忆审计**：记录和扫描所有记忆操作
4. **提示工程**：指示Claude忽略记忆中的指令

有关完整的安全实现和`tests/`中的测试，请参见`memory_tool.py`。

## 下一步

### 资源

- **API文档**：[Claude API参考](https://docs.claude.com/en/api/messages)
- **使用文档**：[记忆工具](https://docs.claude.com/en/docs/agents-and-tools/tool-use/memory-tool)
- **GitHub操作**：[claude-code-action](https://github.com/anthropics/claude-code-action)
- **AWS/Bedrock实现**：[AWS上的Anthropic记忆功能](https://github.com/aws-samples/anthropic-on-aws/tree/main/notebooks/claude_memory_features) - 特定于Bedrock的实现
- **支持**：[support.claude.com](https://support.claude.com)

### 反馈

记忆和上下文管理处于**测试版**。分享您的反馈以帮助我们改进！